In [1]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd
from itertools import combinations
import random

In [2]:
class Neo4jDriver(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def make_req(self, message):
        return self._driver.session().run(message)
    
    def clear(self):  #  DELETE
        return self._driver.session().run('match(n) detach delete (n)')
    
class Experiment(object):
    
    def __init__(self, al_path, l1_path, participants_total, words_total):
        self.al = self.read_table(al_path)[:words_total]
        self.l1 = self.read_table(l1_path)[:words_total]
        self.participants = list(range(participants_total))
        self.l1_distribution, self.translations = self.get_translations(words_total, participants_total)
        self.afc_distribution = self.get_afc()
        self.semantic_test = self.get_semantic()
        
    @staticmethod
    def read_table(path):
        with open(path, 'r', encoding='utf-8') as file:
            return [row for _, row in pd.read_csv(file).iterrows()]
        
    def get_translations(self, total, participants_total):
        all_idxs = list(range(total))
        all_combs = []
        res = []
        i = 0
        while i < participants_total:
            i += 1
            while True:
                tmp = random.sample(all_idxs, k=total)
                if tmp not in all_combs:
                    translation = [self.l1[idx].word for idx in tmp]
                    all_combs.append(tmp)
                    res.append(translation)
                    break
        return all_combs, res
    
    def get_afc(self):
        full_afc = []
        for p_id in self.participants:
            res = {}
            translations = self.translations[p_id]
            combs = list(iter(combinations(translations, 3)))
            for idx, row in enumerate(self.al):
                correct = translations[idx]
                afc = random.choice([comb for comb in combs if correct in comb])
                res[row.word] = (afc, correct)
            full_afc.append(res)
        return full_afc
    
    def get_semantic(self):
        with open('semantic_categories.csv', 'r', encoding='utf-8') as file:
            return {row.word:row.category for _, row in pd.read_csv(file).iterrows()}
    

In [3]:
def add_participant(data, participant, uri, user, password):
    rt = list(range(5, 35))
    ages = list(range(18, 31))
    driver = Neo4jDriver(uri, user, password)
    driver.make_req("create (:Participant {id: '" + str(participant) + "', age: " + str(random.choice(ages)) + "})")
    idxs = data.l1_distribution[participant-1]
    translations = data.translations[participant]
    for idx, i in enumerate(data.al):
        afc_list = data.afc_distribution[participant-1]
        word = i.word
        if idx % 10 == 0:
            driver = Neo4jDriver(uri, user, password)
        j = data.l1[idxs[idx]]
        
        # presentation
        a.make_req("create (:AL {p_id : '" + str(participant) + "', \
                                s_id : '" + str(participant) + '_'+ str(idx) + "', \
                                word : '" + i.word + "'})")
       
        a.make_req("create (:L1 {p_id : '" + str(participant) + "', \
                                s_id : '" + str(participant) + '_'+ str(idx) + "', \
                                word :'" + j.word + "', \
                                gender: '" + j.gender + "', \
                                animacy: '"+ j.anim + "'})")
       
        #  alternative-forced-choice test
        a.make_req("create (:afc  {p_id : '" + str(participant) + "', \
                                s_id : '" + str(participant) + '_'+ str(idx) + "', \
                                a1: '"+ afc_list[word][0][0] + "', \
                                a2: '"+ afc_list[word][0][1] + "', \
                                a3: '"+ afc_list[word][0][2] + "', \
                                correct: '"+ afc_list[word][1] + "', \
                                answer: '" + random.choice(['wrong', 'correct']) + "', \
                                reaction_time: "+ str(random.choice(rt) / 10) + "})")
        
        #  recognition test
        a.make_req("create (:recognition  {p_id : '" + str(participant) + "', \
                                s_id : '" + str(participant) + '_'+ str(idx) + "', \
                                a1: '" + random.choice(translations) + "', \
                                correct: '"+ j.word + "', \
                                answer: '" + random.choice(['wrong', 'correct']) + "', \
                                reaction_time: "+ str(random.choice(rt) / 10) + "})")
        
        #  semantic decision test
        a.make_req("create (:semantics  {p_id : '" + str(participant) + "', \
                                s_id : '" + str(participant) + '_'+ str(idx) + "', \
                                a1: 'люди', \
                                a2: 'животное', \
                                a3: 'предметы', \
                                correct: '"+ data.semantic_test[j.word] + "', \
                                answer: '" + random.choice(['wrong', 'correct']) + "', \
                                reaction_time: "+ str(random.choice(rt) / 10) + "})")
        
        print(participant, idx, i.word, j.word, afc_list[word])
    a.make_req("MATCH (a:Participant),(b:AL) \
                WHERE (b.p_id = a.id AND a.id = '" + str(participant) + "') \
                CREATE (a)<-[r:stimul]-(b) \
                return type(r)")
    a.make_req("MATCH (a:L1),(b:AL) \
                WHERE (a.s_id = b.s_id AND b.p_id = '" + str(participant) + "' ) \
                MERGE (a)<-[r:translate]-(b) \
                RETURN type(r), a.word, b.word")
    a.make_req("MATCH (a:afc),(b:AL) \
                WHERE (a.s_id = b.s_id AND b.p_id = '" + str(participant) + "' ) \
                MERGE (a)<-[r:afc_test]-(b)")
    a.make_req("MATCH (a:recognition),(b:AL) \
                WHERE (a.s_id = b.s_id AND b.p_id = '" + str(participant) + "' ) \
                MERGE (a)<-[r:recognition_test]-(b)")
    a.make_req("MATCH (a:semantics),(b:AL) \
                WHERE (a.s_id = b.s_id AND b.p_id = '" + str(participant) + "' ) \
                MERGE (a)<-[r:semantic_test]-(b)")
        
        

In [4]:
def delete_participant(driver, p_id):
    return driver.make_req("MATCH p =(a)--(b)--(c) \
                            WHERE a.id = b.p_id =  c.p_id = '" + str(p_id) + "' detach delete (p)")

In [5]:
uri = 'bolt://100.27.2.160:33254'
user = 'neo4j'
password = 'brick-capacity-fund'
a = Neo4jDriver(uri, user, password)
data = Experiment(al_path='AL.csv', l1_path='L1.csv', participants_total=8, words_total=5)

In [6]:
a.clear()
for i in range(1, 3):
    add_participant(data, i, uri, user, password)

1 0 adere белка (('белка', 'зебра', 'курица'), 'белка')
1 1 anvugu корова (('белка', 'корова', 'зебра'), 'корова')
1 2 akete зебра (('белка', 'корова', 'зебра'), 'зебра')
1 3 apoktu курица (('корова', 'зебра', 'курица'), 'курица')
1 4 askolte невеста (('белка', 'курица', 'невеста'), 'невеста')
2 0 adere корова (('корова', 'курица', 'белка'), 'корова')
2 1 anvugu невеста (('невеста', 'зебра', 'белка'), 'невеста')
2 2 akete курица (('корова', 'курица', 'зебра'), 'курица')
2 3 apoktu зебра (('корова', 'невеста', 'зебра'), 'зебра')
2 4 askolte белка (('невеста', 'курица', 'белка'), 'белка')


In [7]:
data.al

[word    adere
 Name: 0, dtype: object, word    anvugu
 Name: 1, dtype: object, word    akete
 Name: 2, dtype: object, word    apoktu
 Name: 3, dtype: object, word    askolte
 Name: 4, dtype: object]

In [8]:
r1 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '1' AND c.answer = 'correct' \
            RETURN count(p)").single()

r2 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '1' \
            RETURN count(p)").single()

r3 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' AND c.answer = 'correct' \
            RETURN count(p)").single()

r4 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' \
            RETURN count(p)").single()

print(f"Participant 1: {r1[0]} correct answers out of {r2[0]},\nParticipant 2: {r3[0]} correct answers out of {r4[0]}")

Participant 1: 10 correct answers out of 20,
Participant 2: 11 correct answers out of 20


In [9]:
delete_participant(a, 1)

**COUNT**

In [10]:
a.make_req("match (n:Participant) RETURN count(n)").single()[0]

1

In [11]:
add_participant(data, 1, uri, user, password)

1 0 adere белка (('белка', 'зебра', 'курица'), 'белка')
1 1 anvugu корова (('белка', 'корова', 'зебра'), 'корова')
1 2 akete зебра (('белка', 'корова', 'зебра'), 'зебра')
1 3 apoktu курица (('корова', 'зебра', 'курица'), 'курица')
1 4 askolte невеста (('белка', 'курица', 'невеста'), 'невеста')


In [12]:
r1 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '1' AND c.answer = 'correct' \
            RETURN count(p)").single()

r2 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '1' \
            RETURN count(p)").single()

r3 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' AND c.answer = 'correct' \
            RETURN count(p)").single()

r4 = a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' \
            RETURN count(p)").single()

print(f"Participant 1: {r1[0]} correct answers out of {r2[0]},\nParticipant 2: {r3[0]} correct answers out of {r4[0]}")

Participant 1: 9 correct answers out of 20,
Participant 2: 11 correct answers out of 20


In [13]:
r0 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE y.answer='correct' \
           RETURN count(y)").single()

r1 = a.make_req("MATCH p=(n:Participant)-[r1]-(x)-[r2]-(y) \
            WHERE y.answer='correct' \
           RETURN count(y)").single()

print("Number of correct answers in the semantic test:", r0[0])
print("Number of correct answers in all tests:", r1[0])

Number of correct answers in the semantic test: 6
Number of correct answers in all tests: 20


**Return 3 nodes**

In [14]:
a.make_req("MATCH (n:Participant)-[r1:stimul]-(x)-[r2:afc_test]-(y) \
            RETURN (y)").values()

[[<Node id=71 labels={'afc'} properties={'a1': 'невеста', 'a2': 'курица', 's_id': '2_4', 'a3': 'белка', 'correct': 'белка', 'answer': 'correct', 'reaction_time': 1.9, 'p_id': '2'}>],
 [<Node id=282 labels={'afc'} properties={'a1': 'невеста', 'a2': 'зебра', 'a3': 'белка', 's_id': '2_1', 'correct': 'невеста', 'answer': 'correct', 'reaction_time': 0.9, 'p_id': '2'}>],
 [<Node id=66 labels={'afc'} properties={'a1': 'корова', 'a2': 'курица', 's_id': '2_0', 'a3': 'белка', 'correct': 'корова', 'answer': 'correct', 'reaction_time': 1.7, 'p_id': '2'}>],
 [<Node id=47 labels={'afc'} properties={'a1': 'корова', 'a2': 'невеста', 'a3': 'зебра', 's_id': '2_3', 'correct': 'зебра', 'answer': 'wrong', 'reaction_time': 0.9, 'p_id': '2'}>],
 [<Node id=320 labels={'afc'} properties={'a1': 'корова', 'a2': 'курица', 's_id': '2_2', 'a3': 'зебра', 'correct': 'курица', 'answer': 'correct', 'reaction_time': 3.1, 'p_id': '2'}>],
 [<Node id=103 labels={'afc'} properties={'a1': 'белка', 'a2': 'корова', 'a3': 'зебр

In [15]:
a.make_req("MATCH (n:Participant {id: '1'})-[r1:stimul]-(x)-[r2:afc_test]-(y) \
            RETURN (y.answer)").values()

[['wrong'], ['wrong'], ['correct'], ['correct'], ['correct']]

**AVG**

In [16]:
print("Average age of participants:", a.make_req("MATCH (n:Participant) RETURN avg(n.age)").single()[0])

# Average reaction time of all participants in the alternative-force-choice-test

r2 = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x)-[r2:afc_test]-(y) RETURN avg(y.reaction_time)").single()
r3 = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x)-[r2:semantic_test]-(y) RETURN avg(y.reaction_time)").single()
r4 = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x)-[r2:recognition_test]-(y) RETURN avg(y.reaction_time)").single()

print(f"\nAFC - average reaction time: {r2[0]}\n\n"\
      f"Semantic test - average reaction time: {r3[0]}\n\n" \
      f"Recognition test - average reaction time: {r2[0]}")

Average age of participants: 24.0

AFC - average reaction time: 1.6800000000000002

Semantic test - average reaction time: 2.16

Recognition test - average reaction time: 1.6800000000000002


**Collect**

In [17]:
# Answers of all participants in the alternative-force-choice-test


r1 = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x)-[r2:afc_test]-(y) \
            RETURN collect(y.answer)").single()

print(f"AFC test: {r1[0]}")

AFC test: ['correct', 'correct', 'correct', 'wrong', 'correct', 'wrong', 'wrong', 'correct', 'correct', 'correct']


In [18]:
akete = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x {word: 'akete'})-[r2:semantic_test]-(y) RETURN avg(y.reaction_time)").single()
askolte = a.make_req("MATCH (n:Participant)-[r1:stimul]-(x {word: 'askolte'})-[r2:semantic_test]-(y) RETURN avg(y.reaction_time)").single()

print(f"Semantic test - average reaction time to distinct words:\nakete - {akete[0]},\naskolte - {askolte[0]}")

Semantic test - average reaction time to distinct words:
akete - 2.95,
askolte - 2.8


**MIN**

In [19]:
r1 = a.make_req("MATCH (n:Participant {id: '1'})-[r1:stimul]-(x)-[r2:semantic_test]-(y) RETURN min(y.reaction_time)").single()
r2 = a.make_req("MATCH (n:Participant {id: '2'})-[r1:stimul]-(x)-[r2:semantic_test]-(y) RETURN min(y.reaction_time)").single()

print(f"Semantic test - minimal reaction time:\nParticipant 1 - {r1[0]},\nParticipant 2 - {r2[0]}")

Semantic test - minimal reaction time:
Participant 1 - 0.5,
Participant 2 - 1.5


**MAX**

In [20]:
r1 = a.make_req("MATCH (n:Participant {id: '1'})-[r1:stimul]-(x)-[r2:semantic_test]-(y) RETURN max(y.reaction_time)").single()
r2 = a.make_req("MATCH (n:Participant {id: '2'})-[r1:stimul]-(x)-[r2:semantic_test]-(y) RETURN max(y.reaction_time)").single()

print(f"Semantic test - maximal reaction time:\nParticipant 1 - {r1[0]},\nParticipant 2 - {r2[0]}")

Semantic test - maximal reaction time:
Participant 1 - 3.1,
Participant 2 - 3.3


**ANY**

In [21]:
r1 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.answer='correct') \
           RETURN n.id, x.word, y.answer").values()

print("Words with correct answers in the semantic test:\n", r1)

Words with correct answers in the semantic test:
 [['2', 'askolte', 'correct'], ['2', 'anvugu', 'correct'], ['2', 'adere', 'correct'], ['2', 'akete', 'correct'], ['1', 'anvugu', 'correct'], ['1', 'akete', 'correct']]


In [22]:
# make all the answers in semantic test wrong and check

r1 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.answer='correct') \
           SET y.answer='wrong'") # SET = UPDATE

r2 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.answer='correct') \
           RETURN n.id, x.word, y.answer").values()

r3 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.answer='wrong') \
           RETURN n.id, x.word, y.answer").values()

print("Words with correct answers in the semantic test:\n", r2)
print("Words with wrong answers in the semantic test:\n", r3)

Words with correct answers in the semantic test:
 []
Words with wrong answers in the semantic test:
 [['2', 'askolte', 'wrong'], ['2', 'anvugu', 'wrong'], ['2', 'adere', 'wrong'], ['2', 'apoktu', 'wrong'], ['2', 'akete', 'wrong'], ['1', 'anvugu', 'wrong'], ['1', 'askolte', 'wrong'], ['1', 'adere', 'wrong'], ['1', 'akete', 'wrong'], ['1', 'apoktu', 'wrong']]


**NONE**

In [23]:
r1 = a.make_req("MATCH p=(n:Participant)-[]-()-[]-(y) \
            WHERE NONE (k in nodes (p) WHERE y.answer =  'wrong') \
           RETURN n.id, y.answer, y.reaction_time").values()

r1

[['2', 'correct', 1.4],
 ['2', 'correct', 1.9],
 ['2', 'correct', 0.9],
 ['2', 'correct', 1.7],
 ['2', 'correct', 1.5],
 ['2', 'correct', 3.4],
 ['2', 'correct', 3.1],
 ['1', 'correct', 3.0],
 ['1', 'correct', 0.7],
 ['1', 'correct', 0.6],
 ['1', 'correct', 1.0],
 ['1', 'correct', 3.3],
 ['1', 'correct', 2.4],
 ['1', 'correct', 0.6]]

**OPERATORS**

In [24]:
r1 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.reaction_time < 1) \
           RETURN n.id, x.word, y.answer, y.reaction_time").values()

print(f"Words with reaction time < 1 sec:\n{r1}")

Words with reaction time < 1 sec:
[['1', 'anvugu', 'wrong', 0.5], ['1', 'adere', 'wrong', 0.5]]


In [25]:
r1 = a.make_req("MATCH p=(n:Participant)-[r:stimul]-(x)-[r2:semantic_test]-(y) \
            WHERE ANY (k in nodes (p) WHERE k.reaction_time > 3) \
           RETURN n.id, x.word, y.answer, y.reaction_time").values()

print(f"Words with reaction time > 3 sec:\n{r1}")

Words with reaction time > 3 sec:
[['2', 'akete', 'wrong', 3.3], ['1', 'apoktu', 'wrong', 3.1]]


**What else can I do?**

In [26]:
a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' \
            RETURN labels(c)").values()

[[['semantics']],
 [['L1']],
 [['afc']],
 [['recognition']],
 [['semantics']],
 [['L1']],
 [['afc']],
 [['recognition']],
 [['L1']],
 [['afc']],
 [['recognition']],
 [['semantics']],
 [['semantics']],
 [['L1']],
 [['afc']],
 [['recognition']],
 [['semantics']],
 [['L1']],
 [['afc']],
 [['recognition']]]

In [27]:
a.make_req("MATCH p =(a)--(b)--(c) \
            WHERE a.id = b.p_id =  c.p_id = '2' \
            RETURN relationships(p)").values()[:4]

[[[<Relationship id=0 nodes=(<Node id=285 labels=set() properties={}>, <Node id=57 labels=set() properties={}>) type='stimul' properties={}>,
   <Relationship id=16 nodes=(<Node id=285 labels=set() properties={}>, <Node id=29 labels=set() properties={}>) type='semantic_test' properties={}>]],
 [[<Relationship id=303 nodes=(<Node id=30 labels=set() properties={}>, <Node id=57 labels=set() properties={}>) type='stimul' properties={}>,
   <Relationship id=1 nodes=(<Node id=30 labels=set() properties={}>, <Node id=46 labels=set() properties={}>) type='translate' properties={}>]],
 [[<Relationship id=303 nodes=(<Node id=30 labels=set() properties={}>, <Node id=57 labels=set() properties={}>) type='stimul' properties={}>,
   <Relationship id=6 nodes=(<Node id=30 labels=set() properties={}>, <Node id=47 labels=set() properties={}>) type='afc_test' properties={}>]],
 [[<Relationship id=303 nodes=(<Node id=30 labels=set() properties={}>, <Node id=57 labels=set() properties={}>) type='stimul' pr